In [1]:
# suppress warnings
%pip install stata_setup --quiet
import stata_setup
import sys
sys.path.append(r'C:/Program Files/Stata17/utilities')


Note: you may need to restart the kernel to use updated packages.


In [2]:
from pystata import config
config.init('mp')


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       MP—Parallel Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user 4-core network, expiring 31 May 2023
Serial number: 501709320052
  Licensed to: IFPRI
               IFPRI

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. More than 2 billion observations are allowed; see help obs_advice.
      3. Maximum number of variables is set to 5,000; see help set_maxvar.


In [3]:
import os
## read CML survey dta
import pandas as pd
import datetime
prelim = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\CML_50_Towns_Results\Preliminary Survey\CML  Survey.dta')
## read merged dataset
response = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\CML_50_Towns_Results\CML_Agg_9_2_to_9_16.dta')

response = response.drop(['id_receive','message','id_sent','date_sent','text','time_sent_3'], axis=1)
# mod 10
response['order'] = response['rec_text_order_town'] % 10
# parse date_receive as date
response['date_receive'] = pd.to_datetime(response['date_receive'], format='%m/%d')
# plus 122 years
response['date_receive'] = response['date_receive'] + pd.DateOffset(years=122)

# rename date_receive to date
response.rename(columns={'date_receive':'date'}, inplace=True)
# rename
response.rename(columns={'sender':'receiver_id','text_hr':'time_window_type','order':'n_th_message_received_in_day'}, inplace=True)
# reorder
response = response[['receiver_id','date','time_receive','time_sent','time_window_type','n_th_message_received_in_day','rain','intensity','Zone','Town_name','area_code_woreda','count_message_ind','tot_sent_text_ind','tot_rec_text_ind','tot_miss_text_ind','town_received_text','tot_sent_rec_ind','Longitude','Latitude','POINT_X','POINT_Y']]

In [4]:
## generate 9_2 to 9_16
# format date as YYYY_MM_DD
date = pd.date_range(start='2022-09-02', end='2022-09-16')
date = date.strftime('%Y_%m_%d')

# create a list of dataframes
response_list = []

for i in range(2,17):
    ## read response data
    m = i - 2
    file_path = r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\CML_50_Towns_Results\CML_Respondent_output\\9_' + str(i) +'\\'+date[m]+'_Town.dta'
    # skip if file does not exist
    if not os.path.exists(file_path):
        continue
    response_i = pd.read_stata(file_path)
    ## filter columns sender and time_receive and perc_
    response_i = response_i[['sender','time_receive11','perc_rain_town_11']]
    ## rename columns
    response_i.rename(columns={'sender':'receiver_id','time_receive11':'time_receive','perc_rain_town_11':'perc_rain'}, inplace=True)
    ## add a date column
    response_i['date'] = date[m]
    ## append to list
    response_list.append(response_i)

## merge all dates
response_concated = pd.concat(response_list, ignore_index=True)

## drop if time_receive is missing
response_concated = response_concated.dropna(subset=['time_receive'])
# reformat date as YYYY-MM-DD
response_concated['date'] = pd.to_datetime(response_concated['date'], format='%Y_%m_%d')
#format time_receive as int16
response_concated['time_receive'] = response_concated['time_receive'].astype('int16')

# merge
response = pd.merge(response, response_concated, how='left', on=['receiver_id','date','time_receive'])



In [5]:
prelim = pd.read_stata(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\CML_50_Towns_Results\Preliminary Survey\CML  Survey.dta')
# filter columns of prelim
prelim = prelim[['b1_2','b1_3','b2','b3','b4']]


prelim2 = prelim.copy()
## drop column
prelim2 = prelim2.drop(['b1_2'], axis=1)
prelim = prelim.drop(['b1_3'], axis=1)
# rename columns
prelim.rename(columns={'b1_2':'receiver_id','b2':'age','b3':'gender','b4':'education'}, inplace=True)
prelim2.rename(columns={'b1_3':'receiver_id','b2':'age','b3':'gender','b4':'education'}, inplace=True)

# remove receiver_id with negative values
prelim = prelim[prelim['receiver_id'] > 0]
prelim2 = prelim2[prelim2['receiver_id'] > 0]
# concat
prelim = pd.concat([prelim, prelim2], ignore_index=True)
# remove duplictaes 
prelim = prelim.drop_duplicates(subset=['receiver_id'], keep='first')
prelim[['receiver_id']] = prelim[['receiver_id']]+251000000000
## convert to float64
prelim['receiver_id'] = prelim['receiver_id'].astype('float64')
# merge
response = pd.merge(response, prelim, how='left', on=['receiver_id'])

In [6]:
def parse_time_number(time_number):
    time_decimal = time_number / 100
    hours = int(time_decimal)
    minutes = int((time_decimal - hours) * 100)
    return datetime.time(hours, minutes)

In [7]:
response['time_receive'] = response['time_receive'].apply(parse_time_number)
response['receiver_id'] = response['receiver_id'].astype('int64')
response['time_sent'] = response['time_sent'].apply(parse_time_number)
response['n_th_message_received_in_day'] = response['n_th_message_received_in_day'].astype('int16')

In [12]:
# save response as a utf-8 csv file
response.to_csv(r'C:\Users\WeilunShi\Dropbox (IFPRI)\CML-Ethiopia - team folder\CML_50_Towns_Results\CML_Agg_merged.csv', index=False, encoding='utf-8-sig')
